In [ ]:
#mount ggdrive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import and install tsmixer
!git clone https://github.com/ditschuk/pytorch-tsmixer.git
%cd pytorch-tsmixer
!pip install .

Cloning into 'pytorch-tsmixer'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 48 (delta 21), reused 41 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 19.28 KiB | 9.64 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/pytorch-tsmixer
Processing /content/pytorch-tsmixer
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pytorch-tsmixer: filename=pytorch_tsmixer-0.2.0-py3-none-any.whl size=10459 sha256=87e4c793e2649224a128ec97747bbc9b63d0621afd14d38972edfdaecf7bf51b
  Stored in directory: /root/.cache/pip/wheels/ad/38/42/4625f0ddcd50bd1cd731dfccafc17cf645768f433105317169
Successfully built pytorch-tsmixer


In [20]:
!ls /usr/local/lib/python3.10/dist-packages/torchtsmixer

!cat /usr/local/lib/python3.10/dist-packages/torchtsmixer/layers.py


__init__.py  layers.py	__pycache__  tsmixer_ext.py  tsmixer.py
from __future__ import annotations

from collections.abc import Callable

import torch
import torch.nn.functional as F
from torch import Tensor, nn


class TimeBatchNorm2d(nn.BatchNorm1d):
    """A batch normalization layer that normalizes over the last two dimensions of a
    sequence in PyTorch, mimicking Keras behavior.

    This class extends nn.BatchNorm1d to apply batch normalization across time and
    feature dimensions.

    Attributes:
        num_time_steps (int): Number of time steps in the input.
        num_channels (int): Number of channels in the input.
    """

    def __init__(self, normalized_shape: tuple[int, int]):
        """Initializes the TimeBatchNorm2d module.

        Args:
            normalized_shape (tuple[int, int]): A tuple (num_time_steps, num_channels)
                representing the shape of the time and feature dimensions to normalize.
        """
        num_time_steps, num_channels = n

In [23]:
#Modding the model so it can process the longterm forecast and support debuggin
%%writefile /usr/local/lib/python3.10/dist-packages/torchtsmixer/layers.py

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Callable

class TSMixerExt(nn.Module):
    def __init__(self, sequence_length: int, input_channels: int, output_channels: int, static_channels: int, ff_dim: int, activation_fn: Callable = F.relu, dropout_rate: float = 0.1, normalize_before: bool = False, norm_type: type[nn.Module] = nn.LayerNorm):
        super(TSMixerExt, self).__init__()
        self.sequence_length = sequence_length
        self.prediction_length = sequence_length  # Updated to match sequence_length
        self.input_channels = input_channels
        self.extra_channels = static_channels  # Updated to match static_channels
        self.hidden_channels = ff_dim  # Updated to match ff_dim
        self.static_channels = static_channels
        self.output_channels = output_channels

        # Define layers
        self.fr_static = nn.Linear(static_channels, ff_dim)  # Updated to use ff_dim
        self.fm = nn.Linear(input_channels + ff_dim + static_channels, output_channels)  # Updated to use ff_dim

    def forward(self, x: torch.Tensor, x_static: torch.Tensor, x_extra_hist: torch.Tensor, x_extra_future: torch.Tensor) -> torch.Tensor:
        """Forward pass for the conditional mix layer.

        Args:
            x: A tensor representing dynamic features, typically with shape
               [batch_size, time_steps, input_channels].
            x_static: A tensor representing static features, typically with shape
               [batch_size, static_channels].
            x_extra_hist: A tensor representing extra historical features.
            x_extra_future: A tensor representing extra future features.

        Returns:
            The output tensor after applying time and conditional feature mixing.
        """
        print(f"Inside forward, x shape: {x.shape}")  # Debug: Print shape of x
        print(f"Inside forward, x_static shape: {x_static.shape}")  # Debug: Print shape of x_static

        x = self.time_mixing(x)  # Apply time mixing first
        print(f"Shape after time_mixing: {x.shape}")  # Debug: Shape after time_mixing

        x, _ = self.feature_mixing(x, x_static)  # Apply conditional feature mixing
        print(f"Shape after feature_mixing: {x.shape}")  # Debug: Shape after feature mixing

        return x

def time_to_feature(x: torch.Tensor) -> torch.Tensor:
    """Converts a time series tensor to a feature tensor."""
    return x.permute(0, 2, 1)

feature_to_time = time_to_feature

# Dummy classes for TimeMixing and ConditionalFeatureMixing (for context)
class TimeMixing(nn.Module):
    def __init__(self, sequence_length, input_channels, activation_fn, dropout_rate, norm_type):
        super(TimeMixing, self).__init__()
        # Dummy layer
        self.layer = nn.Linear(sequence_length, input_channels)

    def forward(self, x):
        return F.relu(self.layer(x))

class ConditionalFeatureMixing(nn.Module):
    def __init__(self, sequence_length, input_channels, output_channels, static_channels, ff_dim, activation_fn, dropout_rate, normalize_before, norm_type):
        super(ConditionalFeatureMixing, self).__init__()
        # Dummy layers
        self.layer = nn.Linear(input_channels + static_channels, ff_dim)
        self.output_layer = nn.Linear(ff_dim, output_channels)

    def forward(self, x, x_static):
        v = F.relu(self.layer(torch.cat([x, x_static.unsqueeze(1).expand(-1, x.shape[1], -1)], dim=-1)))
        return self.output_layer(v), v


Overwriting /usr/local/lib/python3.10/dist-packages/torchtsmixer/layers.py


In [24]:
#import thu vien
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from torchtsmixer import TSMixerExt

In [27]:
##short_term forecasting: 5 values
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and preprocess electricity data
def load_electricity_data():
    data_path = '/content/drive/My Drive/Colab Notebooks/dataset/electricity/electricity.csv'
    df = pd.read_csv(data_path)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    return df

# Undersample data
def undersample_data(df, fraction=0.1):
    return df.sample(frac=fraction)

# Normalize data
def normalize_data(df):
    scaler = MinMaxScaler()
    df[df.columns] = scaler.fit_transform(df[df.columns])
    return df, scaler

# Prepare input features
def prepare_data(df, sequence_length=10, prediction_length=5):
    values = df.values.astype(float)
    x_hist, y_target = [], []

    for i in range(len(values) - sequence_length - prediction_length):
        x_hist.append(values[i:i + sequence_length])
        y_target.append(values[i + sequence_length:i + sequence_length + prediction_length])

    x_hist = np.array(x_hist, dtype=np.float32).reshape(-1, sequence_length, 1)
    y_target = np.array(y_target, dtype=np.float32).reshape(-1, prediction_length, 1)

    x_hist = torch.tensor(x_hist, dtype=torch.float32)
    y_target = torch.tensor(y_target, dtype=torch.float32)

    dataset = TensorDataset(x_hist, y_target)
    return dataset

# Define a simple model architecture
class ForecastModel(nn.Module):
    def __init__(self, sequence_length, prediction_length):
        super(ForecastModel, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=32, num_layers=2, batch_first=True)
        self.fc = nn.Linear(32, prediction_length)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

def main():
    drive.mount('/content/drive')
    # Load and preprocess data
    df = load_electricity_data()
    df = undersample_data(df, fraction=0.1)
    df, scaler = normalize_data(df)

    sequence_length = 10
    prediction_length = 5
    batch_size = 128

    dataset = prepare_data(df, sequence_length, prediction_length)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Initialize model, loss, and optimizer
    model = ForecastModel(sequence_length=sequence_length, prediction_length=prediction_length).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training
    num_epochs = 100
    model.train()
    for epoch in range(num_epochs):
        for x_hist_batch, y_target_batch in data_loader:
            x_hist_batch, y_target_batch = x_hist_batch.to(device), y_target_batch.to(device)
            optimizer.zero_grad()
            predictions = model(x_hist_batch)
            loss = criterion(predictions, y_target_batch.squeeze(-1))
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

    # Validation - Predict the first 5 values for demonstration
    model.eval()
    with torch.no_grad():
        for x_hist_batch, y_target_batch in data_loader:
            x_hist_batch = x_hist_batch.to(device)
            predictions = model(x_hist_batch)
            print("Predictions:", predictions.cpu().numpy()[:5])
            print("Actual Values:", y_target_batch.cpu().numpy()[:5])
            break  # Print only the first batch

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch [1/100], Loss: 0.0377
Epoch [2/100], Loss: 0.0414
Epoch [3/100], Loss: 0.0361
Epoch [4/100], Loss: 0.0423
Epoch [5/100], Loss: 0.0402
Epoch [6/100], Loss: 0.0450
Epoch [7/100], Loss: 0.0392
Epoch [8/100], Loss: 0.0425
Epoch [9/100], Loss: 0.0374
Epoch [10/100], Loss: 0.0374
Epoch [11/100], Loss: 0.0389
Epoch [12/100], Loss: 0.0340
Epoch [13/100], Loss: 0.0328
Epoch [14/100], Loss: 0.0385
Epoch [15/100], Loss: 0.0388
Epoch [16/100], Loss: 0.0402
Epoch [17/100], Loss: 0.0339
Epoch [18/100], Loss: 0.0389
Epoch [19/100], Loss: 0.0364
Epoch [20/100], Loss: 0.0370
Epoch [21/100], Loss: 0.0379
Epoch [22/100], Loss: 0.0406
Epoch [23/100], Loss: 0.0358
Epoch [24/100], Loss: 0.0415
Epoch [25/100], Loss: 0.0329
Epoch [26/100], Loss: 0.0324
Epoch [27/100], Loss: 0.0381
Epoch [28/100], Loss: 0.0427
Epoch [29/100], Loss: 0.0420
Epoch [30/100], Loss: 0.0376
Epoch [31/